### Word Group Prepare For Youtube Tool

In [14]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [15]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [16]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [17]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Dutch
Lang Id = 6
File Extention = 6


In [18]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group Prepare For Youtube Tool"

Path(path).mkdir(parents=True, exist_ok=True)

#### Read Data

In [19]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [20]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(700)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,we zijn er dat was het dat was,473,476,nou rik we zijn er dat was het dat was hem,1ZuKeTOZ7IU,https://www.youtube.com/watch?v=1ZuKeTOZ7IU&t=...
1,niet voor het ik maar voor het,4037,4039,dus niet voor het ik maar voor het wij,ufQenq834F4,https://www.youtube.com/watch?v=ufQenq834F4&t=...
2,maar niet als je het niet,793,795,maar niet als je het niet leuk vond,IOMexym5byo,https://www.youtube.com/watch?v=IOMexym5byo&t=...
3,maar niet als je het niet,832,835,maar niet als je het niet leuk vond,2RDPz03xJMg,https://www.youtube.com/watch?v=2RDPz03xJMg&t=...
4,maar niet als je het niet,734,736,maar niet als je het niet leuk vond,3r234qU24b4,https://www.youtube.com/watch?v=3r234qU24b4&t=...
...,...,...,...,...,...,...
575,n van,6,9,n van,zlBTmitsCxM,https://www.youtube.com/watch?v=zlBTmitsCxM&t=6s
576,in de,1229,1234,in de,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...
577,met n,74,76,met n o,YrALyCWSUuU,https://www.youtube.com/watch?v=YrALyCWSUuU&t=74s
578,in de,233,238,in de,IfmgQtst9VI,https://www.youtube.com/watch?v=IfmgQtst9VI&t=...


In [21]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(700)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,dan toch mee omdat ze weet dat de vader anders...,264,270,dan toch mee omdat ze weet dat de vader anders...,knaPebpdfcI,https://www.youtube.com/watch?v=knaPebpdfcI&t=...
1,als ik het niet ben ik zeggen jaar dan heb jij...,2017,2024,lieg als ik het niet ben ik zeggen jaar dan he...,iLJg6UP0y_8,https://www.youtube.com/watch?v=iLJg6UP0y_8&t=...
2,daar is nog wel iets meer over te zeggen maar ...,703,708,daar is nog wel iets meer over te zeggen maar ...,gQPOBsiqB_o,https://www.youtube.com/watch?v=gQPOBsiqB_o&t=...
3,en het komt ook omdat je heel goed moet weten ...,203,210,en het komt ook omdat je heel goed moet weten ...,c5DtYHHKMK0,https://www.youtube.com/watch?v=c5DtYHHKMK0&t=...
4,en toen dacht ik van nou dit moet ik hebben di...,146,155,en toen dacht ik van nou dit moet ik hebben di...,9crP_C_ZvB8,https://www.youtube.com/watch?v=9crP_C_ZvB8&t=...
...,...,...,...,...,...,...
695,waar zij zich aan moeten,518,520,waar zij zich aan moeten houden,LgGiMKoCxlQ,https://www.youtube.com/watch?v=LgGiMKoCxlQ&t=...
696,om daar iets mee te doen,117,119,entiteit om daar iets mee te doen,kgF09oOIJOg,https://www.youtube.com/watch?v=kgF09oOIJOg&t=...
697,kan iedereen mij zien zo,1236,1239,kan iedereen mij zien zo,37RqLFakmo0,https://www.youtube.com/watch?v=37RqLFakmo0&t=...
698,laat ze even weten in de,281,283,laat ze even weten in de comments,kUbE9zaa68Q,https://www.youtube.com/watch?v=kUbE9zaa68Q&t=...


In [22]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(700)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,ik moet het ook gewoon zeggen anders gaat iema...,730,736,ik moet het ook gewoon zeggen anders gaat iema...,LemeIsc8Iy0,https://www.youtube.com/watch?v=LemeIsc8Iy0&t=...
1,dus wanneer je iemand anders ziet eten dat je ...,365,372,dus wanneer je iemand anders ziet eten dat je ...,jSq4KqjykvM,https://www.youtube.com/watch?v=jSq4KqjykvM&t=...
2,als je gewoon mensen bij elkaar aan tafel zet ...,295,304,als je gewoon mensen bij elkaar aan tafel zet ...,7qygIdY0k8Y,https://www.youtube.com/watch?v=7qygIdY0k8Y&t=...
3,nou het valt me niet tegen moet ik zeggen want...,98,104,nou het valt me niet tegen moet ik zeggen want...,URyrZjK46pQ,https://www.youtube.com/watch?v=URyrZjK46pQ&t=98s
4,laten we nou eens kijken wat er gebeurt als er...,576,584,laten we nou eens kijken wat er gebeurt als er...,oewqQ6RkNT8,https://www.youtube.com/watch?v=oewqQ6RkNT8&t=...
...,...,...,...,...,...,...
695,precies d t staat nu onder grote druk,3870,3873,precies d t staat nu onder grote druk,QAjt9liE_a4,https://www.youtube.com/watch?v=QAjt9liE_a4&t=...
696,schrijven jullie willen dit hier gaan,1319,1324,schrijven jullie willen dit hier gaan,F0uYZ4uI47w,https://www.youtube.com/watch?v=F0uYZ4uI47w&t=...
697,ooit heeft iemand zelfs al eens bijna,70,76,ooit heeft iemand zelfs al eens bijna,mF90i2Hk6FU,https://www.youtube.com/watch?v=mF90i2Hk6FU&t=70s
698,gevoel over hebt vertel ja inderdaad,50,52,gevoel over hebt vertel ja inderdaad,EPtpFkfIWec,https://www.youtube.com/watch?v=EPtpFkfIWec&t=50s


In [23]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(700)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,hier slapen alle dieren in rust en wij gaan de...,1251,1273,hier slapen alle dieren in rust en wij gaan de...,iJb2QNlOkpY,https://www.youtube.com/watch?v=iJb2QNlOkpY&t=...
1,nou ik ben in ieder geval heel blij mee en ik ...,1478,1487,nou ik ben in ieder geval heel blij mee en ik ...,4OE321XI9ko,https://www.youtube.com/watch?v=4OE321XI9ko&t=...
2,maar het is wel belangrijk als jullie groeien ...,547,557,maar het is wel belangrijk als jullie groeien ...,k1vwT6_PBi8,https://www.youtube.com/watch?v=k1vwT6_PBi8&t=...
3,maar veel verder kom ik niet want er is in een...,142,152,maar veel verder kom ik niet want er is in een...,jY7Wfl92qis,https://www.youtube.com/watch?v=jY7Wfl92qis&t=...
4,niet te kijken naar het gedrag van de mensen z...,27,35,daar stonden wij een keertje niet te kijken na...,YQh4Xulq0Fo,https://www.youtube.com/watch?v=YQh4Xulq0Fo&t=27s
...,...,...,...,...,...,...
695,wij hebben natuurlijk al even contact gehad ja,392,394,wij hebben natuurlijk al even contact gehad ja,9ahkCAA5m1s,https://www.youtube.com/watch?v=9ahkCAA5m1s&t=...
696,gewoon accepteren wij moeten elkaar accepteren,268,272,gewoon accepteren wij moeten elkaar accepteren,lnsh5XrDsW8,https://www.youtube.com/watch?v=lnsh5XrDsW8&t=...
697,daar eh onderzoek naar laten doen tim ja welke,1230,1232,daar eh onderzoek naar laten doen tim ja welke,zlBTmitsCxM,https://www.youtube.com/watch?v=zlBTmitsCxM&t=...
698,kunnen jullie me daar iets meer over vertellen,89,91,kunnen jullie me daar iets meer over vertellen,sDl3Ze7N_xk,https://www.youtube.com/watch?v=sDl3Ze7N_xk&t=89s


In [24]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(700)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,kijk bijvoorbeeld naar een gebaar als wonen of...,234,244,kijk bijvoorbeeld naar een gebaar als wonen of...,oewqQ6RkNT8,https://www.youtube.com/watch?v=oewqQ6RkNT8&t=...
1,super leuk nu zie je het nog niet heel goed ma...,756,764,super leuk nu zie je het nog niet heel goed ma...,Z0czr5G9r0s,https://www.youtube.com/watch?v=Z0czr5G9r0s&t=...
2,maar veel verder kom ik niet want er is in een...,142,152,maar veel verder kom ik niet want er is in een...,jY7Wfl92qis,https://www.youtube.com/watch?v=jY7Wfl92qis&t=...
3,en ja dat is wel gewoon iets wat je toch ondan...,229,237,en ja dat is wel gewoon iets wat je toch ondan...,5S5QX4ZK3Q0,https://www.youtube.com/watch?v=5S5QX4ZK3Q0&t=...
4,ik kan me nu ook wel een beetje voorstellen ho...,166,174,ik kan me nu ook wel een beetje voorstellen ho...,c3kFCnzQdHs,https://www.youtube.com/watch?v=c3kFCnzQdHs&t=...
...,...,...,...,...,...,...
695,jij weet natuurlijk heel goed wie jouw toekoms...,133,135,jij weet natuurlijk heel goed wie jouw toekoms...,iSTwyPpsZaA,https://www.youtube.com/watch?v=iSTwyPpsZaA&t=...
696,oke even terug naar nut wanneer heeft iets nou...,155,159,oke even terug naar nut wanneer heeft iets nou...,K8SPryAFk7M,https://www.youtube.com/watch?v=K8SPryAFk7M&t=...
697,veel mensen jongen iedereen komt kijken natuur...,194,197,veel mensen jongen iedereen komt kijken natuur...,2iTVYSEsdZc,https://www.youtube.com/watch?v=2iTVYSEsdZc&t=...
698,regelmatig wordt daar door partijen over gespr...,708,713,regelmatig wordt daar door partijen over gespr...,7wyEAK7STy8,https://www.youtube.com/watch?v=7wyEAK7STy8&t=...


In [25]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(700)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,dus als het gaat om de slecht nieuws gesprekke...,1110,1121,dus als het gaat om de slecht nieuws gesprekke...,QPALQDtz7so,https://www.youtube.com/watch?v=QPALQDtz7so&t=...
1,op moment van bedenken is het blijkbaar niet z...,333,347,op moment van bedenken is het blijkbaar niet z...,xKhL2kGvszM,https://www.youtube.com/watch?v=xKhL2kGvszM&t=...
2,en dan kun je wel zeggen daar heb ik eigenlijk...,1876,1887,en dan kun je wel zeggen daar heb ik eigenlijk...,37RqLFakmo0,https://www.youtube.com/watch?v=37RqLFakmo0&t=...
3,in ieder geval hartstikke bedankt ik denk dat ...,611,621,in ieder geval hartstikke bedankt ik denk dat ...,yD5aDc5N5qc,https://www.youtube.com/watch?v=yD5aDc5N5qc&t=...
4,wij hadden op dinsdag contact die dag daarna z...,386,394,wij hadden op dinsdag contact die dag daarna z...,LemeIsc8Iy0,https://www.youtube.com/watch?v=LemeIsc8Iy0&t=...
...,...,...,...,...,...,...
695,gemaakt waar handicap geen rol speelde over vr...,1426,1432,gemaakt waar handicap geen rol speelde over vr...,DniD4L9cACc,https://www.youtube.com/watch?v=DniD4L9cACc&t=...
696,mooie achtergrond toch heel goed absoluut zeke...,134,136,mooie achtergrond toch heel goed absoluut zeke...,zl1IPpX3sok,https://www.youtube.com/watch?v=zl1IPpX3sok&t=...
697,zou jij ons bedrijf willen aanraden aan iemand...,686,689,zou jij ons bedrijf willen aanraden aan iemand...,HI3fG8o4W7g,https://www.youtube.com/watch?v=HI3fG8o4W7g&t=...
698,wij gaan vandaag eigenlijk ons producten laten...,139,142,wij gaan vandaag eigenlijk ons producten laten...,i12kdg53FyE,https://www.youtube.com/watch?v=i12kdg53FyE&t=...


In [26]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(700)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,het zijn geen net zoals met de voice of wat wi...,517,529,het zijn geen net zoals met de voice of wat wi...,bBYvUrZhWXs,https://www.youtube.com/watch?v=bBYvUrZhWXs&t=...
1,dus als het gaat om de slecht nieuws gesprekke...,1110,1121,dus als het gaat om de slecht nieuws gesprekke...,QPALQDtz7so,https://www.youtube.com/watch?v=QPALQDtz7so&t=...
2,op moment van bedenken is het blijkbaar niet z...,333,347,op moment van bedenken is het blijkbaar niet z...,xKhL2kGvszM,https://www.youtube.com/watch?v=xKhL2kGvszM&t=...
3,het is inderdaad mogelijk om bij elkaar te bli...,261,270,het is inderdaad mogelijk om bij elkaar te bli...,7D5ZiQsUy48,https://www.youtube.com/watch?v=7D5ZiQsUy48&t=...
4,en afhankelijk van waar je woont in nederland ...,1103,1112,en afhankelijk van waar je woont in nederland ...,gQPOBsiqB_o,https://www.youtube.com/watch?v=gQPOBsiqB_o&t=...
...,...,...,...,...,...,...
695,are you ready for it yes i was born ready thre...,235,240,are you ready for it yes i was born ready thre...,EomE4PhFc0A,https://www.youtube.com/watch?v=EomE4PhFc0A&t=...
696,dit echt super ja zeker wil jij nou zon m line...,107,112,dit echt super ja zeker wil jij nou zon m line...,yS5YLewAo_I,https://www.youtube.com/watch?v=yS5YLewAo_I&t=...
697,belangrijke vragen moeten stellen alvorens dan...,90,95,belangrijke vragen moeten stellen alvorens dan...,EILHq0Gx82Y,https://www.youtube.com/watch?v=EILHq0Gx82Y&t=90s
698,waarin hij klassieke muziek heel toegankelijk ...,208,213,waarin hij klassieke muziek heel toegankelijk ...,bDObx1fpdmA,https://www.youtube.com/watch?v=bDObx1fpdmA&t=...


#### End Time Reassign

In [27]:
df_28["end_time"] = df_28["start_time"] + 1
df_200["end_time"] = df_200["start_time"] + 1
df_1000["end_time"] = df_1000["start_time"] + 1
df_5000["end_time"] = df_5000["start_time"] + 1
df_10000["end_time"] = df_10000["start_time"] + 1
df_20000["end_time"] = df_20000["start_time"] + 1
df_40000["end_time"] = df_40000["start_time"] + 1

In [28]:
out_path = "/home/kurubal/Downloads"

In [29]:
df_28.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_200.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_1000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_5000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_10000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_20000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_40000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)

In [ ]:
# !!!!! End Time Reassign files result check on youtube tool and change file with youtube tool file result as same name (old file should remove) !!!!! 

#### End Time Reassign Convert Orjinal (From Youtube Tool Result)

In [ ]:
df_28_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28_video_list = list(set(df_28_youtube_tool["video_id"]))

In [ ]:
df_200_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200_video_list = list(set(df_200_youtube_tool["video_id"]))

In [ ]:
df_1000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000_video_list = list(set(df_1000_youtube_tool["video_id"]))

In [ ]:
df_5000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000_video_list = list(set(df_5000_youtube_tool["video_id"]))

In [ ]:
df_10000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000_video_list = list(set(df_10000_youtube_tool["video_id"]))

In [ ]:
df_20000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000_video_list = list(set(df_20000_youtube_tool["video_id"]))

In [ ]:
df_40000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000_video_list = list(set(df_40000_youtube_tool["video_id"]))

##### Video List In video_id 

In [ ]:
df_28_new = df_28[df_28["video_id"].isin(df_28_video_list)]
#df_28_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv", index=False)
df_28_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_28_new

In [ ]:
df_200_new = df_200[df_200["video_id"].isin(df_200_video_list)]
#df_200_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv", index=False)
df_200_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_200_new

In [ ]:
df_1000_new = df_1000[df_1000["video_id"].isin(df_1000_video_list)]
#df_1000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv", index=False)
df_1000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_1000_new

In [ ]:
df_5000_new = df_5000[df_5000["video_id"].isin(df_5000_video_list)]
#df_5000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv", index=False)
df_5000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_5000_new

In [ ]:
df_10000_new = df_10000[df_10000["video_id"].isin(df_10000_video_list)]
#df_10000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv", index=False)
df_10000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_10000_new

In [ ]:
df_20000_new = df_20000[df_20000["video_id"].isin(df_20000_video_list)]
#df_20000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv", index=False)
df_20000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_20000_new

In [ ]:
df_40000_new = df_40000[df_40000["video_id"].isin(df_40000_video_list)]
#df_40000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv", index=False)
df_40000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)
df_40000_new

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_*_Word{file_ext}.*")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass